In [ ]:
multiplier = 16

In [2]:
from repalette.datasets import PairRecolorDataset

In [3]:
data = PairRecolorDataset(multiplier, shuffle_palette=False, sort_palette=True)

In [4]:
# _, data = data.split(test_size=0.001, shuffle=True)

In [5]:
train, test = data.split(test_size=0.2, shuffle=True)
test, val = test.split(test_size=0.5, shuffle=True)

In [6]:
from repalette.datasets.utils import ShuffleDataLoader

train_dataloader = ShuffleDataLoader(train, shuffle=False, num_workers=8, batch_size=8)

In [7]:
val_dataloader = ShuffleDataLoader(val, shuffle=False, num_workers=8, batch_size=8)

In [8]:
test_dataloader = ShuffleDataLoader(test, shuffle=False, num_workers=8, batch_size=8)

In [9]:
len(train)

595200

In [10]:
from repalette.models import PaletteNet

In [11]:
import torch
import torch.nn as nn
import pytorch_lightning as pl

In [12]:
from repalette.models import PaletteNet

In [13]:
from repalette.constants import S3_MODEL_CHECKPOINTS_DIR, S3_LIGHTNING_LOGS_DIR, S3_BUCKET_NAME

In [14]:
# from s3fs import S3FileSystem
# fs = S3FileSystem(S3_BUCKET_NAME)

In [15]:
experiment_name = "PaletteNet-sort"

In [16]:
import os

In [17]:
# !export S3_REGION="US-EAST-2"

In [18]:
from dotenv import load_dotenv

In [19]:
# load aws credentials

load_dotenv()

True

In [20]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from repalette.constants import DEFAULT_LR, DEFAULT_BETA_1, DEFAULT_BETA_2
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
import os


checkpoint_callback = ModelCheckpoint(
    dirpath=os.path.join(S3_MODEL_CHECKPOINTS_DIR, experiment_name),
    monitor="Val/Loss",
    verbose=True,
    mode='min',
#     save_top_k=1
)

early_stop_callback = EarlyStopping(
   monitor="Val/Loss",
   min_delta=0.00,
   patience=20,
   verbose=False,
   mode='min'
)

hparams={
    'lr': DEFAULT_LR / 10, "beta_1": 0.9, "beta_2": 0.99
}
model = PaletteNet(
    train_dataloader,
    val_dataloader,
    test_dataloader,
    hparams=hparams,
)
logger = WandbLogger(
    name=experiment_name, project="repalette"
)
trainer = Trainer(
    gpus=1, # use gpu!
    logger=logger,
    callbacks=[early_stop_callback],
#     checkpoint_callback=checkpoint_callback,
#     auto_lr_find="learning_rate"
)

# trainer.tune(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [ ]:
trainer.fit(model)

wandb: Currently logged in as: danielgafni (use `wandb login --relogin` to force relogin)



  | Name               | Type              | Params
---------------------------------------------------------
0 | feature_extractor  | FeatureExtractor  | 11.0 M
1 | recoloring_decoder | RecoloringDecoder | 2.9 M 
2 | loss_fn            | MSELoss           | 0     
